**Based on the work** https://aliaksandrsiarohin.github.io/first-order-model-website/

In [1]:
#@title Run this cell to configure Deepfake
 
#download model
!pip install -U --no-cache-dir gdown --pre

download_ids = ['1Tl_7LBCvpry4mnO4HUysZt4PD389faAk',
                '10Jq0Y7Tj-D2bjhbcMKJpG1Qgqeul9Eel', 
                '1sCLqI2z2oR03TrPKp6pGxo3SY33uAbsn', 
                '11WkiZOhhu_m55xH1KS2cPTeYKK-LOYJ8', 
                '16nA653uE1Fth23G22hJ2J8C0KnrHMoh7',
                '1HmzFfWQHa9Q6h4UTsgiRHyy4l8iaz-5z',
                '1nAMOTARuggKRkPQy0BV2nkRTRJtr-mtN',
                '17K-iqIqKpkzxz-GRk5xrdeZWalObI2c3',
                '148HHyBTBm4Dh3GJwQ3g6r-0jXn_AkH9g',
                '1KilV716g2IP1JnbMv-8nNZXcBU8-J3zZ']

for id in download_ids:
  print("Attept download from", id)
  response = !gdown --id $id --output deepfake_data.tar.gz
  if response[0] == 'Downloading...':
    break
  else:
    continue
  
  
#extract this file
!tar -xzvf deepfake_data.tar.gz
#download repo
!git clone https://github.com/AliaksandrSiarohin/first-order-model

!git clone https://github.com/AwaleSajil/Celeb_pot.git
#go inside this folder
%cd first-order-model
 
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from IPython.display import Image
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")
 
 
def display(source, driving, generated1=None, generated2 = None):
    fig = plt.figure(figsize=(8 + 4 * (generated1 is not None) + 4 * (generated2 is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated1 is not None:
            cols.append(generated1[i])
        if generated2 is not None:
            cols.append(generated2[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
 
Checkpoint = 'vox-cpk.pth' #@param["vox-cpk.pth", "vox-adv-cpk.pth"]
checkpoint_p = {
    "vox-cpk.pth": '/content/first-order-motion-model/vox-cpk.pth.tar', 
    "vox-adv-cpk.pth": '/content/first-order-motion-model/vox-adv-cpk.pth.tar'
}
 

def show_local_mp4_video(file_name, slowmoFactor, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''
                        <script>
                          var vid = document.getElementById("myVideo");
                          vid.playbackRate = 1/{3};
                        </script>
                        <video width="{0}" height="{1}" alt="test" id="myVideo" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii'), slowmoFactor))
  
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=str(checkpoint_p[Checkpoint]))
 
 
from demo import make_animation
from skimage import img_as_ubyte
# from moviepy.editor import *
print("CONFIGURE DONE")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.2.0-py3-none-any.whl size=14273 sha256=176f0909602150430f76e7aed29e4b93052837fbcbe6b1a34508261d070ce46b
  Stored in directory: /tmp/pip-ephem-wheel-cache-g75ww2ph/wheels/8c/17/ff/58721d1fabdb87c21a0529948cf39e2be9af90ddbe4ad65944
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
Attept download from 1Tl_7LBCvpry4mnO4HUysZt4PD389faAk
first-order-motion-model/
first-order-motion-model/cartoons-01.png
first-order-motion-model/03.png
first-order-motion-model/07.mkv
first-order-motion-model/02.png
first-order-motion-model/doll-07.png
first-order-motion-model/got-01.png
first-order-motion-model/doll-01.png
first-order-motion-model/got-02.png
first-order-motion-model/05.png
first-order-motion-model/statue

In [2]:
#@title Test Deepfake with <u>Source Image</u> and <u>Driving Video</u> from sample data<br> <h6>Execute this to view results</h6>
 
Source_Image = 'Vladimir Putin - President of Russia 2018' #@param ["Statue", "Mona Lisa", "GOT-Daenerys", "GOT-Tyrion_Lannister", "GOT-villian", "Doll", "Shrek", "Scarlett Johansson", "Angelina Jolie", "Jennifer Lawrence", "Emma Stone", "Tom Cruise", "Will Smith", "Robert Downey", "Bratt Pit", "Beyoncé", "Lady Gaga", "Justin Bieber", "Katy Perry", "Taylor Swift", "Amitabh Bachchan", "Shahrukh Khan", "Akshay Kumar", "Aamir Khan", "Irrfan Khan", "Ranbir Kapoor", "Rishi Kapoor", "Priyanka Chopra", "Deepika Padukone", "Shraddha Kapoor", "Alia Bhatt", "Anushka Sharma", "Katrina Kaif", "Jacqueline Fernandez", "Manita Devkota Miss Nepal 2018", "Laxmi Prasad Devkota", "Buddha", "Ronald Reagan - 40th president of the US", "Vladimir Putin - President of Russia 2018", "Theresa Mary - Prime Minister of the UK 2016", "Trump", "Obma", "Xi Jinping", "Elon Musk", "Bill Gates", "Kim Jong-un", "Trump 2", "Benedict Cumberbatch", "Groot Guardians of the Galaxy", "Rocket - Guardians of the Galaxy", "Chris Evans", "Chris Hemsworth", "Albert Einstein", "Steve Jobs", "Mark Zuckerberg", "Osama bin Laden", "Hitler", "Stephen Amell"]
Driving_Video = 'Trump' #@param ["Obama", "Leonardo_Dicaprio", "Trump", "Guy"] 
source_path = {
    "Statue": '/content/first-order-motion-model/statue-02.png', 
    "Mona Lisa": '/content/first-order-motion-model/05.png',
    "GOT-Daenerys": '/content/first-order-motion-model/got-05.png', 
    "GOT-Tyrion_Lannister": '/content/first-order-motion-model/got-07.png', 
    "GOT-villian": '/content/first-order-motion-model/got-01.png', 
    "Doll": '/content/first-order-motion-model/doll-03.png', 
    "Shrek": '/content/first-order-motion-model/cartoons-01.png',
    "Scarlett Johansson": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.13.16 PM.png',
    "Angelina Jolie": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.14.04 PM.png',
    "Jennifer Lawrence": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.14.46 PM.png',
    "Emma Stone": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.15.10 PM.png',
    "Tom Cruise": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.16.24 PM.png',
    "Will Smith": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.17.46 PM.png',
    "Robert Downey": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.18.12 PM.png',
    "Bratt Pit": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.19.33 PM.png',
    "Beyoncé": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.20.41 PM.png',
    "Lady Gaga": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.21.03 PM.png',
    "Justin Bieber": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.21.32 PM.png',
    "Katy Perry": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.22.22 PM.png',
    "Taylor Swift": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.22.56 PM.png',
    "Amitabh Bachchan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.23.58 PM.png',
    "Shahrukh Khan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.24.21 PM.png',
    "Akshay Kumar": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.24.40 PM.png',
    "Aamir Khan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.25.10 PM.png',
    "Irrfan Khan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.25.33 PM.png',
    "Ranbir Kapoor": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.26.18 PM.png',
    "Rishi Kapoor": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.26.26 PM.png',
    "Priyanka Chopra": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.26.54 PM.png',
    "Deepika Padukone": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.27.21 PM.png',
    "Shraddha Kapoor": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.27.48 PM.png',
    "Alia Bhatt": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.28.13 PM.png',
    "Anushka Sharma": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.28.55 PM.png',
    "Katrina Kaif": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.29.25 PM.png',
    "Jacqueline Fernandez": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.30.21 PM.png',
    "Manita Devkota Miss Nepal 2018": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.32.04 PM.png',
    "Laxmi Prasad Devkota": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.33.03 PM.png',
    "Buddha": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.33.57 PM.png',
    "Ronald Reagan - 40th president of the US": '/content/first-order-motion-model/01.png',
    "Vladimir Putin - President of Russia 2018": '/content/first-order-motion-model/02.png',
    "Theresa Mary - Prime Minister of the UK 2016": '/content/first-order-motion-model/03.png',
    "Trump":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.20.01 PM.png',
    "Obma":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.20.39 PM.png',
    "Xi Jinping":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.21.52 PM.png',
    "Elon Musk": '/content/Celeb_pot/Screen Shot 2020-06-17 at 3.23.10 PM.png',
    "Bill Gates":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.23.44 PM.png',
    "Kim Jong-un":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.35.59 PM.png',
    "Trump 2": '/content/Celeb_pot/Screen Shot 2020-06-17 at 4.26.39 PM.png',
    "Benedict Cumberbatch": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.44.05 PM.png',
    "Groot Guardians of the Galaxy": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.43.27 PM.png',
    "Rocket - Guardians of the Galaxy": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.43.00 PM.png',
    "Chris Evans": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.42.10 PM.png',
    "Chris Hemsworth": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.41.45 PM.png',
    "Albert Einstein": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.39.36 PM.png',
    "Steve Jobs": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.39.11 PM.png',
    "Mark Zuckerberg": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.38.38 PM.png',
    "Osama bin Laden": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.37.57 PM.png',
    "Hitler": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.37.19 PM.png',
    "Stephen Amell": '/content/Celeb_pot/Screen Shot 2020-06-17 at 6.22.55 PM.png',

}
 
driving_path = {
    "Obama": '/content/first-order-motion-model/00.mp4', 
    "Leonardo_Dicaprio": '/content/first-order-motion-model/10.mp4', 
    "Trump": '/content/first-order-motion-model/04.mp4', 
    "Guy": '/content/first-order-motion-model/08.mp4'
}
source_image = imageio.imread(str(source_path[Source_Image]))
driving_video = imageio.mimread(str(driving_path[Driving_Video]))
 
#Resize image and video to 256x256
 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
predictions1 = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
#save resulting video
# imageio.mimsave('../generated_expression.mp4', [img_as_ubyte(frame) for frame in predictions1])
#video can be downloaded from /content folder
predictions2 = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions1, predictions2).to_html5_video())

100%|██████████| 211/211 [00:38<00:00,  5.46it/s]


In [10]:
#@title Deepfake with your own data <h5>Upload source image and driving video (that are face cropped) then copy the paths to fields below</h5> <h5>You may use the source and driving video from sample as well</h5> <h6>Execute this to view results</h6>

Source_Image = 'Vladimir Putin - President of Russia 2018' #@param ["Statue", "Mona Lisa", "GOT-Daenerys", "GOT-Tyrion_Lannister", "GOT-villian", "Doll", "Shrek", "Scarlett Johansson", "Angelina Jolie", "Jennifer Lawrence", "Emma Stone", "Tom Cruise", "Will Smith", "Robert Downey", "Bratt Pit", "Beyoncé", "Lady Gaga", "Justin Bieber", "Katy Perry", "Taylor Swift", "Amitabh Bachchan", "Shahrukh Khan", "Akshay Kumar", "Aamir Khan", "Irrfan Khan", "Ranbir Kapoor", "Rishi Kapoor", "Priyanka Chopra", "Deepika Padukone", "Shraddha Kapoor", "Alia Bhatt", "Anushka Sharma", "Katrina Kaif", "Jacqueline Fernandez", "Manita Devkota Miss Nepal 2018", "Laxmi Prasad Devkota", "Buddha", "Ronald Reagan - 40th president of the US", "Vladimir Putin - President of Russia 2018", "Theresa Mary - Prime Minister of the UK 2016", "Trump", "Obma", "Xi Jinping", "Elon Musk", "Bill Gates", "Kim Jong-un", "Trump 2", "Benedict Cumberbatch", "Groot Guardians of the Galaxy", "Rocket - Guardians of the Galaxy", "Chris Evans", "Chris Hemsworth", "Albert Einstein", "Steve Jobs", "Mark Zuckerberg", "Osama bin Laden", "Hitler", "Stephen Amell"]{allow-input: true}
Driving_Video = 'Sample_video' #@param ["Obama", "Leonardo_Dicaprio", "Trump", "Guy"] {allow-input: true}
source_path = {
    "My_Sample": '/content/first-order-motion-model/photo.jpg',
    "Statue": '/content/first-order-motion-model/statue-02.png', 
    "Mona Lisa": '/content/first-order-motion-model/05.png',
    "GOT-Daenerys": '/content/first-order-motion-model/got-05.png', 
    "GOT-Tyrion_Lannister": '/content/first-order-motion-model/got-07.png', 
    "GOT-villian": '/content/first-order-motion-model/got-01.png', 
    "Doll": '/content/first-order-motion-model/doll-03.png', 
    "Shrek": '/content/first-order-motion-model/cartoons-01.png',
    "Scarlett Johansson": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.13.16 PM.png',
    "Angelina Jolie": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.14.04 PM.png',
    "Jennifer Lawrence": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.14.46 PM.png',
    "Emma Stone": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.15.10 PM.png',
    "Tom Cruise": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.16.24 PM.png',
    "Will Smith": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.17.46 PM.png',
    "Robert Downey": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.18.12 PM.png',
    "Bratt Pit": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.19.33 PM.png',
    "Beyoncé": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.20.41 PM.png',
    "Lady Gaga": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.21.03 PM.png',
    "Justin Bieber": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.21.32 PM.png',
    "Katy Perry": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.22.22 PM.png',
    "Taylor Swift": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.22.56 PM.png',
    "Amitabh Bachchan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.23.58 PM.png',
    "Shahrukh Khan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.24.21 PM.png',
    "Akshay Kumar": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.24.40 PM.png',
    "Aamir Khan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.25.10 PM.png',
    "Irrfan Khan": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.25.33 PM.png',
    "Ranbir Kapoor": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.26.18 PM.png',
    "Rishi Kapoor": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.26.26 PM.png',
    "Priyanka Chopra": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.26.54 PM.png',
    "Deepika Padukone": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.27.21 PM.png',
    "Shraddha Kapoor": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.27.48 PM.png',
    "Alia Bhatt": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.28.13 PM.png',
    "Anushka Sharma": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.28.55 PM.png',
    "Katrina Kaif": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.29.25 PM.png',
    "Jacqueline Fernandez": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.30.21 PM.png',
    "Manita Devkota Miss Nepal 2018": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.32.04 PM.png',
    "Laxmi Prasad Devkota": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.33.03 PM.png',
    "Buddha": '/content/Celeb_pot/Screen Shot 2020-06-17 at 12.33.57 PM.png',
    "Ronald Reagan - 40th president of the US": '/content/first-order-motion-model/01.png',
    "Vladimir Putin - President of Russia 2018": '/content/first-order-motion-model/02.png',
    "Theresa Mary - Prime Minister of the UK 2016": '/content/first-order-motion-model/03.png',
    "Trump":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.20.01 PM.png',
    "Obma":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.20.39 PM.png',
    "Xi Jinping":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.21.52 PM.png',
    "Elon Musk": '/content/Celeb_pot/Screen Shot 2020-06-17 at 3.23.10 PM.png',
    "Bill Gates":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.23.44 PM.png',
    "Kim Jong-un":'/content/Celeb_pot/Screen Shot 2020-06-17 at 3.35.59 PM.png',
    "Trump 2": '/content/Celeb_pot/Screen Shot 2020-06-17 at 4.26.39 PM.png',
    "Benedict Cumberbatch": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.44.05 PM.png',
    "Groot Guardians of the Galaxy": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.43.27 PM.png',
    "Rocket - Guardians of the Galaxy": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.43.00 PM.png',
    "Chris Evans": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.42.10 PM.png',
    "Chris Hemsworth": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.41.45 PM.png',
    "Albert Einstein": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.39.36 PM.png',
    "Steve Jobs": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.39.11 PM.png',
    "Mark Zuckerberg": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.38.38 PM.png',
    "Osama bin Laden": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.37.57 PM.png',
    "Hitler": '/content/Celeb_pot/Screen Shot 2020-06-17 at 5.37.19 PM.png',
    "Stephen Amell": '/content/Celeb_pot/Screen Shot 2020-06-17 at 6.22.55 PM.png',

}

driving_path = {
    "Obama": '/content/first-order-motion-model/00.mp4', 
    "Leonardo_Dicaprio": '/content/first-order-motion-model/10.mp4', 
    "Trump": '/content/first-order-motion-model/04.mp4', 
    "Guy": '/content/first-order-motion-model/08.mp4',
    "Sample_video": '/content/first-order-motion-model/2021-12-25-135115_ZJ7CBpst.mp4'
}

try:
  s_path = str(source_path[Source_Image])
except:
  s_path = str(Source_Image)

try:
  d_path = str(driving_path[Driving_Video])
except:
  d_path = str(Driving_Video)


source_image = imageio.imread(s_path)
driving_video = imageio.mimread(d_path, memtest=False)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


#find the frame_rate of driving video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate $d_path
frame_rate_source = float(Fraction(frame_rate_source[0]))


# check if video has audio stream
audio_ = !ffprobe -i $d_path -show_streams -select_streams a -loglevel error
audio_stream = list(audio_)
hasAudio = None


if len(audio_stream) != 0:
  # has audio
  hasAudio = True
else:
  hasAudio = False


#save resized img and video
t1= imageio.imwrite('/content/resized_source_img.png', source_image)
t2 = imageio.mimsave('/content/resized_driving_video.mp4', driving_video , fps = int(frame_rate_source))



#extarct audio from driving video
if hasAudio:
  !ffmpeg -i $d_path '/content/df_audio.mp3'


predictions1 = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
imageio.mimsave('../temp1_yourDeepfake1.mp4', [img_as_ubyte(frame) for frame in predictions1], fps = int(frame_rate_source))

predictions2 = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
imageio.mimsave('../temp1_yourDeepfake2.mp4', [img_as_ubyte(frame) for frame in predictions2], fps = int(frame_rate_source))

#combine 2 vidoes
!ffmpeg \
  -i /content/resized_source_img.png \
  -i /content/resized_driving_video.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  -y \
  /content/all_input_vid.mp4


if hasAudio:
  #combile 3 videos
  !ffmpeg -i '/content/all_input_vid.mp4' -i '/content/temp1_yourDeepfake1.mp4' -i '/content/temp1_yourDeepfake2.mp4' -filter_complex "[1:v][0:v]scale2ref=oh*mdar:ih[1v][0v];[2:v][0v]scale2ref=oh*mdar:ih[2v][0v];[0v][1v][2v]hstack=3,scale='2*trunc(iw/2)':'2*trunc(ih/2)'" -y /content/final.mp4

else:
  #combile 3 videos
  !ffmpeg -i '/content/all_input_vid.mp4' -i '/content/temp1_yourDeepfake1.mp4' -i '/content/temp1_yourDeepfake2.mp4' -filter_complex "[1:v][0:v]scale2ref=oh*mdar:ih[1v][0v];[2:v][0v]scale2ref=oh*mdar:ih[2v][0v];[0v][1v][2v]hstack=3,scale='2*trunc(iw/2)':'2*trunc(ih/2)'" -y /content/yourDeepfake1.mp4


#concatenating the video and audio
if hasAudio:
  !ffmpeg -y  -i /content/final.mp4 -i /content/df_audio.mp3 -c:v copy -c:a aac /content/yourDeepfake1.mp4

!cp /content/yourDeepfake1.mp4 /content/yourDeepfake2.mp4


clear_output()
#display video
show_local_mp4_video('/content/yourDeepfake1.mp4', 1, width=1024, height=256)
